In [20]:
import logging
import numpy as np
import pandas as pd

logging.basicConfig()

logger = logging.getLogger('revtsov.hw3')
logger.setLevel(logging.INFO)

#### Please Note: You can use the same function to calculate results for problems 3.4 and 3.5. 

In [26]:
def _portfolio_value(X, S, S0, k, p, q, u, d, r, n, expiry_type, option_type):
    """
    Helper recursive function
    """
    # calculate portfolio value
    Xn_m1 = (1/(1+r)) * (p * X[:-1] + q * X[1:])
    # calculate shares
    Dn_m1 = (X[:-1] - X[1:]) / (S[:-1] - S[1:])
    i = np.array(range(n+1))
    # price
    Sn_m1 = S0 * u**(n - i) * d**i
    # option payoff
    psi_S_m1 = np.maximum(Sn_m1 - k, 0) if option_type == 'call' else np.maximum(k - Sn_m1, 0)
    Vn_m1 = np.maximum(Xn_m1, psi_S_m1) if expiry_type == 'american' else Xn_m1.copy()

    logger.info(f'X({n}): {Xn_m1}')
    logger.info(f'S({n}): {Sn_m1}')
    logger.info(f'psi({n}): {psi_S_m1}')
    logger.info(f'V({n}): {Vn_m1}')

    if Vn_m1.shape[0] == 1:
        return Vn_m1, Dn_m1
    else:
        return _portfolio_value(Vn_m1, Sn_m1, S0, k, p, q, u, d, r, n-1, expiry_type, option_type)

    
def binomial_pricing(S0: float, T: int, K: float, r: float, sigma: float, N: int, option_type: str, expiry_type: str):
    """
    Price and hedge European and American put and call options
    
    :param S0: Initial stock price
    :param T: Maturity date
    :param K: Strike price
    :param r: Risk free rate of interest
    :param sigma: Volatility
    :param N: Steps in binomial tree
    :param option_type: Option type: call or put option
    :param expiry_type: Option expiry type: american or european
    
    :return X0: Value of option/replicating portfolio
    :return delta: Number of shares to hedge
    """
    
    # assert inputs
    assert option_type in ['call', 'put']
    assert expiry_type in ['american', 'european']    

    # calculate u/d values
    u = 1.2#np.exp(sigma*np.sqrt(T/N))
    d = 0.8#np.exp(-1*sigma*np.sqrt(T/N))
    # find p and q
    p  = ((1 + r) - d) / (u - d)
    q = (1 - p)

    # determine the starting condition with leafs of the binomal tree
    i = np.array(range(N+1))
    # price at N
    Sn = S0 * u**(N - i) * d**i
    # portfolio value = option payoff
    Xn = np.maximum(Sn - K, 0) if option_type == 'call' else np.maximum(K - Sn, 0)

    logger.info(f'Starting X: {Xn}')
    logger.info(f'Starting S: {Sn}')
    
    # call recursive function to find portfolio value and number of shares level by level
    X0, d0 = _portfolio_value(Xn, Sn, S0, K, p, q, u, d, r, N-1, expiry_type, option_type)
    logger.info(f'Value: {X0}, shares: {d0}')
    return X0, d0

### Example Call:

In [27]:
X0, d0 = binomial_pricing(S0=50, T=1, K=52, r=0.051, sigma=0.1, N=2, option_type='put', expiry_type='american')

INFO:revtsov.hw3:Starting X: [ 0.  4. 20.]
INFO:revtsov.hw3:Starting S: [72. 48. 32.]
INFO:revtsov.hw3:X(1): [1.41769743 9.47668887]
INFO:revtsov.hw3:S(1): [60. 40.]
INFO:revtsov.hw3:psi(1): [ 0. 12.]
INFO:revtsov.hw3:V(1): [ 1.41769743 12.        ]
INFO:revtsov.hw3:X(0): [5.09952915]
INFO:revtsov.hw3:S(0): [50.]
INFO:revtsov.hw3:psi(0): [2.]
INFO:revtsov.hw3:V(0): [5.09952915]
INFO:revtsov.hw3:Value: [5.09952915], shares: [-0.52911513]


In [28]:
print(f'{X0}, {d0}')

[5.09952915], [-0.52911513]
